In [23]:
# pdp package path
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

# imports
import random
import time
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [24]:
## Define problems with sizes 10, 15, 20
# Each with an id 1, 2, 3, ... where (id // 3) * 5 + 10 = number of requests

possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

# 2 small compartments and 2 large compartments
small_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 2 large compartments
medium_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 3 large compartments
large_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),  
    ]
)

vehicles = [small_vehicle, medium_vehicle, large_vehicle]

group_size = 3
instance_sizes = [20]
problem_ids = list(range(group_size * len(instance_sizes)))

# create random problems with seed based on id!
problems = []
modelled_problems = []
for prob_id in problem_ids:
    instance_size_idx = prob_id // group_size
    instance_size = instance_sizes[instance_size_idx]
    nb_items = instance_size
    vehicle = small_vehicle
    V = list(range(0, nb_items * 2 + 1))
    random.seed(prob_id)
    items = random.choices(possible_items, k=nb_items)
    random.seed(prob_id)
    distance_matrix = [
        [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V]
        for j in V
    ]
    
    problem = TwoDimensionalProblem(items, vehicle, distance_matrix, name=f"prob_{prob_id}")
    problems.append(problem)

    # Modellel Problem for time-limited gurobi
    modelled_problem = ModelledTwoDimensionalProblem(items, vehicle, distance_matrix=distance_matrix)
    modelled_problem.create_model()
    modelled_problem.apply_constraints()
    modelled_problem.set_model_objective()
    modelled_problems.append(modelled_problem)


In [25]:
# Initial, quick solutions by gurobi
initial_solutions_dict = {}
for prob_id, problem in enumerate(modelled_problems):
    initial_solution_time_limit = 3 # 1 sec for 10, 2 for 15, 3 for 20  
    modelled_problem = modelled_problems[prob_id]
    modelled_problem.solve(time_limit=initial_solution_time_limit)
    initial_solutions_dict[prob_id] = modelled_problem.extract_solution()

Set parameter TimeLimit to value 3
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 15447 rows, 2150 columns and 74727 nonzeros
Model fingerprint: 0x669357f7
Variable types: 0 continuous, 2150 integer (1945 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 1162 rows and 308 columns
Presolve time: 0.27s
Presolved: 14285 rows, 1842 columns, 53994 nonzeros
Variable types: 0 continuous, 1842 integer (1642 binary)
Found heuristic solution: objective 2181.0000000

Root relaxation: objective 1.416261e+02, 1339 iterations, 0.28 seconds (0.36 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  141.62612    0  141 2181.00000  141.626

In [26]:
# Convergence 
time_limit = 10
rel_problem_ids = [0, 1, 2]
nb_processes_list = [1, 3, 6]

lns_conv_solvers = dict()
for prob_id in rel_problem_ids:
    problem = problems[prob_id]
    init_sol = initial_solutions_dict[prob_id]
    # min_d, max_d = (0.15, 0.15) if prob_id == 6 else (0.2, 0.2)
    min_d, max_d = (0.15, 0.15)

    prob_solvers = []
    for nb_process in nb_processes_list:
        plns = PLNS(problem, init_sol, time_limit=time_limit, nb_processes=nb_process, 
                    destroy_strategy=DestroyStrategy.HIGHEST_COST, 
                    min_destruction_degree=min_d, max_destruction_degree=max_d,
                    )

        plns.set_destruction_degree_criterion(DestructionDegreeCriterion.CONSTANT)
        prob_solvers.append(plns)

    lns_conv_solvers[prob_id] = prob_solvers


Parallel LNS initialized with 1 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 6 processes
Parallel LNS initialized with 1 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 6 processes
Parallel LNS initialized with 1 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 6 processes


In [27]:
for prob_id in rel_problem_ids:
    for solver in lns_conv_solvers[prob_id]:
        solver.search()

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 36 iterations
Total time: 10.00 Best objective: 400 found in 22 iterations in 6.08 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 29 iterations
Total time: 10.00 Best objective: 446 found in 7 iterations in 2.53 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 23 iterations
Total time: 10.00 Best objective: 333 found in 8 iterations in 4.71 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 33 iterations
Total time: 10.00 Best objective: 643 found in 5 iterations in 1.62 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 32 iterations
Total time: 10.00 Best objective: 572 found in 6 iterations in 2.44 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 19 iterations
Total time: 10.00 Best objective: 572 found in 7 iterati

In [28]:
def get_solver_stats(solver):
    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    stats = dict(
        objectives=objectives,
        iterations=iterations,
        times_found=time_found,
        improving_iterations=improving_iterations,
        improving_objectives=improving_objectives,
        improving_times=improving_times,
    )

    return stats

In [103]:
# Define colors for processes,more processes darker shade

# proc_colors = ["#BCD2E8", "#91BAD6", "#73A5C6", "#528AAE", "#2E5984", "#1E3F66"]

# light = "#64a0c8"
# normal = "#0065bd"
# dark = "#005293"

light = "#62BDE7"
normal = "#0D87BF"
dark = "#0E3344"

proc_colors = {
    1: light,
    3: normal,
    6: dark,
}

fig = make_subplots(
    rows=3,
    cols=1,
    y_title="Objective",
    x_title="Time(s)",
    shared_yaxes=True,
    shared_xaxes=True,
    subplot_titles=("Problem 1","Problem 2", "Problem 3"),
    vertical_spacing=0.075,
)

show_all = True

for prob_id in rel_problem_ids:
    r = prob_id + 1
    problem_solvers = lns_conv_solvers[prob_id]

    best = None
    best_solvers = []
    for solver in problem_solvers:
        nb_process = solver.nb_processes
        stats = get_solver_stats(solver)

        if not best or solver.best_cached_solution.objective < best.objective:
            best = solver.best_cached_solution
            best_solvers = [solver]
        elif solver.best_cached_solution.objective == best.objective:
            best_solvers.append(solver)

        # improving solutions
        fig.add_trace(go.Scatter(
            x=stats["improving_times"],
            y=stats["improving_objectives"],
            name=f"{nb_process} process(es) (impro)",
            marker_color=proc_colors[nb_process],
            mode="lines+markers",
        ),
                      row=r,
                      col=1)

        if show_all:
            # all solutions
            fig.add_trace(go.Scatter(
                x=stats["times_found"],
                y=stats["objectives"],
                name=f"{nb_process} process(es) (all)",
                opacity=0.5,
                marker_color=proc_colors[nb_process],
                mode="lines",
            ),
                          row=r,
                          col=1)

        # fig.update_yaxes(title_text="Objective", row=r, col=1)
        fig.update_xaxes(
            # title_text="Time (s)",
                         row=r,
                         col=1,
                         nticks=time_limit + 1)

    # highlight best solutions
    for solver in best_solvers:
        best_solver = solver
        best = solver.best_cached_solution
        fig.add_trace(
            go.Scatter(
                x=[best.time_to_find],
                y=[best.objective],
                mode="markers+text",
                text=[f"({'{:.2f}'.format(best.time_to_find)}, {best.objective})"],
                textposition="bottom center",
                name=f"problem {prob_id + 1} best",
                opacity=0.7,
                marker=dict(
                    size=14,
                    line=dict(width=2,
                              color=proc_colors[best_solver.nb_processes]),
                    color=proc_colors[best_solver.nb_processes],
                ),
                showlegend=False,
            ),
            row=r,
            col=1,
        )
    #     fig.add_shape(type="circle",
    #     xref="x", yref="y",
    #     x0=best.time_to_find, y0=best.objective,
    #     line_color="LightSeaGreen",
    # )

# for prob_id in rel_problem_ids:
#     r = prob_id + 1
#     problem_solvers = lns_conv_solvers[prob_id]

#     for solver in problem_solvers:
#         nb_process = solver.nb_processes
#         stats = get_solver_stats(solver)

#         # improving solutions
#         fig.add_trace(go.Scatter(x=stats["improving_iterations"],
#                         y=stats["improving_objectives"],
#                         name=f"{nb_process} process(es)(impro)",
#                         marker_color=proc_colors[nb_process],
#                         mode="lines+markers",
#                         ),
#             row=r,
#             col=2)

#         if show_all:
#             # all solutions
#             fig.add_trace(go.Scatter(x=stats["iterations"],
#                                 y=stats["objectives"],
#                                 name=f"{nb_process} process(es)(all)",
#                                 opacity=0.5,
#                                 marker_color=proc_colors[nb_process],
#                                 mode="lines",
#                                 ),
#                     row=r,
#                     col=2)

#         fig.update_yaxes(title_text="Objective", row=r, col=2)
#         fig.update_xaxes(title_text="Iteration Number", row=r, col=2)

names = set()
fig.for_each_trace(lambda trace: trace.update(showlegend=False)
                   if (trace.name in names) else names.add(trace.name))

# fig.update_layout(height=1000, width=1200, title_text="Objective Value as the LNS Progresses")
fig.update_layout(height=1000,
                  width=800,
                  title_text="Objective Value for Fast Convergence PLNS")
# fig.update_layout(title_text="Objective Value as the LNS Progresses")